<a href="https://colab.research.google.com/github/ricardobizerra/custom-mlp/blob/main/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IF867 - Introdução à Aprendizagem Profunda
### 1ª atividade prática

Discente(s): Ricardo Bizerra de Lima Filho (rblf)

Período: 2024.2

## Instruções e Requisitos
- Objetivo: Implementar e treinar um Multilayer Perceptron (MLP), inteiramente em [NumPy](https://numpy.org/doc/stable/) ou [Numba](https://numba.readthedocs.io/en/stable/index.html), sem o uso de bibliotecas de aprendizado profundo.
- A atividade pode ser feita em dupla.

### Tarefas

__Implementação (50%):__

- Construa um MLP com uma camada de entrada, pelo menos duas camadas ocultas e uma camada de saída.
- Implemente pelo menos duas funções de ativação diferentes para as camadas ocultas; use Sigmoid e Linear para a camada de saída.
- Implemente forward e backpropagation.
- Implemente um otimizador de sua escolha, adequado ao problema abordado.
- Implemente as funções de treinamento e avaliação.

__Aplicação (30%):__

  Teste se os seus modelos estão funcionando bem com as seguintes tarefas:
  - Regressão
  - Classificação binária

__Experimentação (20%):__

  Teste os seus modelos com variações na arquitetura, no pré-processamento, etc. Escolha pelo menos uma das seguintes opções:
  - Variações na inicialização de pesos
  - Variações na arquitetura
  - Implementação de técnicas de regularização
  - Visualização das ativações e gradientes

***Bônus:*** Implemente o MLP utilizando uma biblioteca de machine learning (ex.: [PyTorch](https://pytorch.org/), [TensorFlow](https://www.tensorflow.org/?hl=pt-br), [tinygrad](https://docs.tinygrad.org/), [Jax](https://jax.readthedocs.io/en/latest/quickstart.html)) e teste-o em uma das aplicações e em um dos experimentos propostos. O bônus pode substituir um dos desafios de aplicação ou experimentos feitos em NumPy, ou simplesmente somar pontos para a pontuação geral.

### Datasets recomendados:
Aqui estão alguns datasets recomendados, mas fica a cargo do aluno escolher os datasets que utilizará na atividade, podendo escolher um dataset não listado abaixo.
- Classificação

  - [Iris](https://archive.ics.uci.edu/dataset/53/iris)
  - [Breast Cancer Wisconsin (Diagnostic)](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic)
  - [CDC Diabetes Health Indicators](https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators)

- Regressão

  - [Air Quality](https://archive.ics.uci.edu/dataset/360/air+quality)
  - [Student Performance](https://archive.ics.uci.edu/dataset/320/student+performance)
  - [Wine Quality](https://archive.ics.uci.edu/dataset/186/wine+quality)

### Requisitos para Entrega

Um notebook Jupyter (de preferência, o link do colab) ou script Python contendo:

- Código: Implementação completa da MLP.
- Gráficos e Análises: Gráficos da curva de perda, ativações, gradientes e insights do treinamento, resultantes dos experimentos com parada antecipada e diferentes técnicas de regularização.
- Relatório: Um breve relatório detalhando o impacto de várias configurações de hiperparâmetros(ex.: inicialização de pesos, número de camadas ocultas e neurônios) e métodos de regularização no desempenho do modelo.


In [ ]:
!pip install ucimlrepo

In [ ]:
import numpy as np
from ucimlrepo import fetch_ucirepo
import math
from sklearn.model_selection import train_test_split

In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
# Dataset para Classificação Binária: Breast Cancer Wisconsin

breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

X_classification = breast_cancer_wisconsin_diagnostic.data.features.values
X_classification = X_classification.T

y_classification = breast_cancer_wisconsin_diagnostic.data.targets
y_classification = (y_classification["Diagnosis"] == "B").astype(int)
y_classification = y_classification.values
y_classification = y_classification.reshape(1, -1)

In [ ]:
# Dataset para Regressão: Student Performance

student_performance = fetch_ucirepo(id=320)

X_regression = student_performance.data.features.values.T
y_regression = student_performance.data.targets.values

In [ ]:
# Funções de ativação

def activation_sigmoid(x):
  return 1 / (1 + np.exp(-x))

def activation_tanh(x):
  return np.tanh(x)

def activation_relu(x):
  return np.maximum(0, x)

def activation_linear(x):
  return x

activation_functions = {
    "sigmoid": activation_sigmoid,
    "tanh": activation_tanh,
    "relu": activation_relu,
    "linear": activation_linear
}

In [ ]:
# Derivada de função de ativação

def derivative_sigmoid(x):
    sig = activation_sigmoid(x)
    return sig * (1 - sig)

def derivative_tanh(x):
    return 1 - np.tanh(x) ** 2

def derivative_relu(x):
    return np.where(x > 0, 1, 0)

def derivative_linear(x):
    return 1

derivatives = {
    "sigmoid": derivative_sigmoid,
    "tanh": derivative_tanh,
    "relu": derivative_relu,
    "linear": derivative_linear
}

In [ ]:
class Perceptron:
  def __init__(self, bias, inputs, weights, activation):
    self.bias = bias
    self.inputs = inputs
    self.weights = weights
    self.activation_function = activation_functions[activation]

    if len(inputs) != len(weights):
      raise Exception(f'Inputs and weights array shall have the same length. Inputs length: {len(inputs)}, weights length: {len(weights)}')

  def forward(self):
    initial_result = self.bias + (self.inputs * self.weights)
    result = self.activation_function(initial_result)

    return result

In [ ]:
class Layer:
  def __init__(self, type, bias, inputs, weights, activation, size):
    self.type = type
    self.bias = bias
    self.inputs = inputs
    self.weights = weights
    self.activation = activation
    self.size = size

    self.perceptrons = [
      Perceptron(
        bias=self.bias[i],
        inputs=self.inputs[i],
        weights=self.weights[i],
        activation=self.activation
      ) for i in range(size)
    ]

  def forward(self):
    layer_result = []

    for perceptron in self.perceptrons:
      perceptron_result = perceptron.forward()
      layer_result.append(perceptron_result)

    return layer_result

In [ ]:
class MLP:
  def __init__(self, n_classes, hidden_layer_sizes, activation, learning_rate):
    self.hidden_layer_sizes = hidden_layer_sizes
    self.activation = activation
    self.learning_rate = learning_rate
    self.n_classes = n_classes

    self.weights = []

  def forward(self, inputs):
    n_features = len(inputs)
    layer_sizes = [n_features] + self.hidden_layer_sizes + [self.n_classes]

    self.weights += [np.random.randn(n) for n in layer_sizes]
    print(self.weights)

    for layer in range(len(layer_sizes)):
      n_perceptrons = layer_sizes[layer]
      print(f'{n_perceptrons} for layer {layer + 1}')
      for perceptron in range(n_perceptrons):
        i = 0

In [ ]:
mlp = MLP(hidden_layer_sizes=[2, 4, 2], activation="sigmoid", learning_rate=0.1, n_classes=1)
mlp.forward(X_classification)

[array([ 0.19686124,  0.73846658,  0.17136828, -0.11564828, -0.3011037 ,
       -1.47852199, -0.71984421, -0.46063877,  1.05712223,  0.34361829,
       -1.76304016,  0.32408397, -0.38508228, -0.676922  ,  0.61167629,
        1.03099952,  0.93128012, -0.83921752, -0.30921238,  0.33126343,
        0.97554513, -0.47917424, -0.18565898, -1.10633497, -1.19620662,
        0.81252582,  1.35624003, -0.07201012,  1.0035329 ,  0.36163603]), array([-0.64511975,  0.36139561]), array([ 1.53803657, -0.03582604,  1.56464366, -2.6197451 ]), array([0.8219025 , 0.08704707]), array([-0.29900735])]
30 for layer 1
2 for layer 2
4 for layer 3
2 for layer 4
1 for layer 5
